In [1]:
import numpy as np
import numpy.ma as ma
from numpy import arange,array,ones,linalg

from mpl_toolkits import mplot3d

import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = False
from matplotlib import cm, colors
from matplotlib.patches import Ellipse
import matplotlib as mpl

from scipy.optimize import minimize
from scipy import ndimage
from scipy.ndimage import zoom, gaussian_filter1d
from scipy import stats

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.nddata import bitmask
from astropy.nddata.bitmask import BitFlagNameMap

import math

from tqdm import tqdm

import copy

import os

from Cythonformult import potentialenergycalc

from sympy.interactive import printing
printing.init_printing(use_latex = True)

import warnings

In [2]:
current_directory = os.getcwd()
print(current_directory)

/global/u1/k/kadglass/DESI_SGA/Morphology


In [3]:
#SGA_Param_Data = Table.read('SGA_Asym_Data.fits')
# SGA_Param_Data = Table.read('SGA_M20_Data1.fits')
SGA_Param_Data = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA_Param_Data[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [4]:
# Bits in the dict
MASKBITS = dict(
    NPRIMARY   = 0x1,   # not PRIMARY
    BRIGHT     = 0x2,
    SATUR_G    = 0x4,
    SATUR_R    = 0x8,
    SATUR_Z    = 0x10,
    ALLMASK_G  = 0x20,
    ALLMASK_R  = 0x40,
    ALLMASK_Z  = 0x80,
    WISEM1     = 0x100, # WISE masked
    WISEM2     = 0x200,
    BAILOUT    = 0x400, # bailed out of processing
    MEDIUM     = 0x800, # medium-bright star
    GALAXY     = 0x1000, # SGA large galaxy
    CLUSTER    = 0x2000, # Cluster catalog source
    SATUR_I    = 0x4000,
    ALLMASK_I  = 0x8000,
    SUB_BLOB   = 0x10000,
)

In [5]:
def makecompactimage(mmc_image, mask_c):
    # extract the image shape and coordinates
    rows, cols = mmc_image.shape
    y, x = np.mgrid[0:rows, 0:cols]
    squared_distance = x**2 + y**2
    
    # flatten the images
    flattened = np.ravel(mmc_image)
    
    # sort the indices by assosiated intensity pixel, greatest to lowest intensity
    sorted_pix = np.argsort(flattened)[::-1]

    # make a list of intensity values, replacing masked values with 0
    filledflattened = flattened.filled(0.0)[sorted_pix]

    # sort the indices replaced with zeros masked intensity, greatest to lowest
    sorted_filled_pix = np.argsort(filledflattened)[::-1]

    # sort the intensities of the replaced masked values
    filledintlist = filledflattened[sorted_filled_pix]
    
    # find the index of the brightest pixel in the original image
    brightest_pixel_index = np.unravel_index(np.argmax(mmc_image, axis=None), mmc_image.shape)

    # calculate squared distances from the center (brightest pixel)
    squared_distances_centered = (x - brightest_pixel_index[1])**2 + (y - brightest_pixel_index[0])**2

    # sort the pixels based on squared distances from the center
    sorted_distances_centered = np.argsort(squared_distances_centered.flatten())

    # create a circular arrangement around the brightest pixel
    arranged_image = np.zeros_like(mmc_image)
    arranged_image.flat[sorted_distances_centered] = filledintlist

    compact_image = ma.array(arranged_image, mask=mask_c)
    
    return compact_image

def potentialenergycalc(mmc_image):
    '''Note: this function is using too much memory and needs to be rewritten.'''
    
    # get pixel coordinates for non-masked pixels of image
    y, x = np.where(~mmc_image.mask)
    
    # rij_matrix where each element represents the distance between the corresponding pixel pairs
    # distance = sqrt{ (x_2 - x_1)^2 + (y_2 - y_1)^2
    rij_matrix = np.sqrt((x[:, np.newaxis] - x)**2 + (y[:, np.newaxis] - y)**2)
    print('*** rij_matrix:', rij_matrix.shape)
    flux_matrix = mmc_image[y, x]
    print('*** flux_matrix:', flux_matrix.shape)
    
    # apply the mask and sum the potential energy
    potential_energy = 0.5*(np.sum(flux_matrix[np.newaxis, :] * flux_matrix[:, np.newaxis] / rij_matrix))
    return potential_energy

def potentialenergycalc(mmc_image):
    # Get pixel coordinates for non-masked pixels of image
    y, x = np.where(~mmc_image.mask)
    
    # Initialize potential energy
    potential_energy = 0.0
    
    # Calculate potential energy using double for loop
    for i in range(len(x)):
        for j in range(len(x)):
            if i != j:
                # Calculate distance between pixel pairs
                rij = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2)
                # Add contribution to potential energy
                potential_energy += 0.5 * (mmc_image[y[i], x[i]] * mmc_image[y[j], x[j]]) / rij
    
    return potential_energy

In [6]:
def multcalc(compact_potetnial_energy, actual_potential_energy):
    multiplicity = 100*(np.log10(compact_potetnial_energy / actual_potential_energy))
    return multiplicity

In [8]:
warnings.filterwarnings("ignore", category=UserWarning)

# Create a list to store indices of processed galaxies
processed_galaxies = []
no_bitmask = []
no_image_path = []
image_shape_unknown = []

n_galaxies = len(SGA_Param_Data)

SGA_Param_Data['Mult_g'] = np.NaN * np.ones(n_galaxies)
SGA_Param_Data['Mult_r'] = np.NaN * np.ones(n_galaxies)
SGA_Param_Data['Mult_z'] = np.NaN * np.ones(n_galaxies)

galaxy_range = range(85, 88)

for i in tqdm(galaxy_range, desc="Processing Galaxies", unit="galaxy", leave=True):
    f1 = math.trunc(SGA_Param_Data['RA'][i])
    f2 = math.trunc((1000*SGA_Param_Data['RA'][i]))
    f3 = math.trunc((100000*SGA_Param_Data['RA'][i]))
    f4 = 'm' if SGA_Param_Data['DEC'][i] < 0 else 'p'
    f5 = abs(math.trunc((100000*SGA_Param_Data['DEC'][i])))
    path = '/pscratch/sd/b/barnerg/SGA2024/cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5)
    
        
    if os.path.exists(path):
        fits_file = fits.open(path, ignore_missing_simple=True, ignore_missing_end=True)

        try:
            fits_file[1].data
        except:
            no_bitmask.append(path)
            var_exists = False
            fits_file.close()
        else:
            var_exists = True

        if var_exists:
            image = fits_file[0].data
            bit_mask = fits_file[1].data
            fits_file.close()
            
            print('*** Image:', image.shape, image.dtype)
            print('*** Bitmask:', bit_mask.shape, bit_mask.dtype)
            
            # reshape the data
            # checking to see if the galaxy has 3 color channels
            if image.shape[0] == 3:
                # the tuple swaps the 1st and 3rd dimensions, resulting in a new array, and assigns this array to the variable image_data
                image_data = np.transpose(image, (1, 2, 0))
                #print("image_data shape:", image_data.shape)
                
                mg_image = ma.array(image_data[:,:,0])
                mr_image = ma.array(image_data[:,:,1])
                mz_image = ma.array(image_data[:,:,2])
                
                # Generate bitmask to apply to image
                condition = bit_mask & MASKBITS['GALAXY'] != 0
                mask_ = condition.astype(int)

                # Generate bitmask to apply to image
                condition2 = mask_ & (MASKBITS['BRIGHT'] | MASKBITS['SATUR_G'] | MASKBITS['SATUR_R'] | MASKBITS['SATUR_Z'] | MASKBITS['WISEM1'] | MASKBITS['WISEM2']) != 0
                mask_2 = mask_ & ~condition2

                # make the mask (318, 318, 3), where the third dimension represents the same 2D mask array duplicated three times
                mask = np.dstack([mask_2.astype(bool), mask_2.astype(bool), mask_2.astype(bool)])
                mimage_data = ma.array(image_data, mask=~mask)

                # masked color images
                mmg_image = ma.array(image_data[:,:,0], mask=~mask_2.astype(bool))
                mmr_image = ma.array(image_data[:,:,1], mask=~mask_2.astype(bool))
                mmz_image = ma.array(image_data[:,:,2], mask=~mask_2.astype(bool))

                # Use the masks from the original images
                mask_g = mmg_image.mask
                mask_r = mmr_image.mask
                mask_z = mmz_image.mask
                
                # call makecompactimage for each channel
                compactimageg = makecompactimage(mmg_image, mask_g)
                compactimager = makecompactimage(mmr_image, mask_r)
                compactimagez = makecompactimage(mmz_image, mask_z)
                
                mmg_image = mmg_image.byteswap().newbyteorder()
                print(type(mmg_image))
                print(mmg_image.dtype)
                
                # call potentialenergycalc for each channel to calculate actual and compact potential
                actualpotentialenergyg = potentialenergycalc(mmg_image.data.astype(np.float64), 
                                                             mmg_image.mask.astype(np.int64))
                actualpotentialenergyr = potentialenergycalc(mmr_image.data.astype(np.float64), 
                                                             mmr_image.mask.astype(np.int64))
                actualpotentialenergyz = potentialenergycalc(mmz_image.data.astype(np.float64), 
                                                             mmz_image.mask.astype(np.int64))
                
                compactpotentialenergyg = potentialenergycalc(compactimageg.data.astype(np.float64), 
                                                              compactimageg.mask.astype(np.int64))
                compactpotentialenergyr = potentialenergycalc(compactimager.data.astype(np.float64), 
                                                              compactimager.mask.astype(np.int64))
                compactpotentialenergyz = potentialenergycalc(compactimagez.data.astype(np.float64), 
                                                              compactimagez.mask.astype(np.int64))
                
                # call multcalc
                multiplicityg = multcalc(compactpotentialenergyg, actualpotentialenergyg)
                multiplicityr = multcalc(compactpotentialenergyr, actualpotentialenergyr)
                multiplicityz = multcalc(compactpotentialenergyz, actualpotentialenergyz)
                
                # append values
                SGA_Param_Data['Mult_g'][i] = multiplicityg
                SGA_Param_Data['Mult_r'][i] = multiplicityr
                SGA_Param_Data['Mult_z'][i] = multiplicityz

                # Append the index of the processed galaxy to the list
                processed_galaxies.append(i)
                print("Galaxy processed!", i, path)

            else:
                image_shape_unknown.append(path)
    else: 
        no_image_path.append(path)

Processing Galaxies:   0%|          | 0/3 [00:00<?, ?galaxy/s]

*** Image: (3, 213, 213) >f4
*** Bitmask: (213, 213) >i4
<class 'numpy.ma.core.MaskedArray'>
float32


Processing Galaxies:  33%|███▎      | 1/3 [00:00<00:00,  3.88galaxy/s]

Galaxy processed! 85 /pscratch/sd/b/barnerg/SGA2024/cutouts/190/190297/19029721p6607370.fits
*** Image: (3, 253, 253) >f4
*** Bitmask: (253, 253) >i4
<class 'numpy.ma.core.MaskedArray'>
float32


Processing Galaxies:  67%|██████▋   | 2/3 [00:01<00:01,  1.05s/galaxy]

Galaxy processed! 86 /pscratch/sd/b/barnerg/SGA2024/cutouts/120/120157/12015780p3717878.fits
*** Image: (3, 445, 445) >f4
*** Bitmask: (445, 445) >i4
<class 'numpy.ma.core.MaskedArray'>
float32


Processing Galaxies: 100%|██████████| 3/3 [00:30<00:00, 10.27s/galaxy]

Galaxy processed! 87 /pscratch/sd/b/barnerg/SGA2024/cutouts/193/193707/19370744p5888232.fits


In [ ]:
print(f"Galaxies with no bitmask: {len(no_bitmask)}")
print(f"Galaxies with no image path: {len(no_image_path)}")
print(f"Galaxies with an unknown image shape: {len(image_shape_unknown)}")
print(f"Galaxies successfully processed: {len(processed_galaxies)}")
print(f"All Galaxies Processed: {len(no_bitmask) + len(no_image_path) + len(image_shape_unknown) + len(processed_galaxies)}")
print(f"Total Galaxies: {len(SGA_Param_Data)}")

In [ ]:
print(SGA_Param_Data)